In [1]:
import os
import tensorflow as tf
import pandas as pd


2023-02-24 13:47:19.483654: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-24 13:47:23.738225: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-02-24 13:47:28.897733: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.7/lib64:/usr/local/cuda-11.7/lib64:
2023-02-24 13:47:28.918227: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; 

In [40]:
import tensorflow as tf
import multiprocessing

from google.cloud import storage


class Dataset:
    def __init__(self, tfrecords_path, epochs, batch_size):
        self.tfrecords_path = tfrecords_path
        self.epochs = epochs
        self.batch_size = batch_size

    def list_files(self):
        return [os.path.join(tfrecords_path,file_path) for file_path in os.listdir(tfrecords_path)]

    def build(self):
        files = self.list_files()

        print("build_tf record: files_count: {} / batch_size: {} / epochs: {}".format(len(files), self.batch_size, self.epochs))

        ds = tf.data.TFRecordDataset(files, num_parallel_reads=multiprocessing.cpu_count())
        
        '''''
            Shuffle and reapeat
        '''''
        
        
        ds = ds.shuffle(buffer_size=1024 * 1 * 10)
        ds = ds.repeat(count=self.epochs)
        
        
        
        '''''
            Map and batch
        '''''
        
                      
        ds = ds.map(self.__parse__, num_parallel_calls=None)
        ds = ds.batch(self.batch_size,drop_remainder=False)
        
        
                      
        ds = ds.prefetch(buffer_size=5)
                      

        return ds
    
   
    @staticmethod
    def __parse__(example):
        parsed = tf.parse_single_example(example, features={
            'emb' : tf.io.FixedLenFeature([], tf.string),
            'label' : tf.io.FixedLenSequenceFeature([], tf.int64, allow_missing=True)
        })
        
        content = tf.io.parse_single_example(element, data)

        label = tf.cast(content['label'], tf.int32)
        label_hot = tf.one_hot(label1[0], label1[1])
        
        emb = content['emb']
        #get our 'feature'
        feature = tf.io.parse_tensor(emb, out_type=tf.float32)

        inp = {'emb': feature}
        out = {'global_output': label_hot}

        return inp, out


In [31]:

def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))): # if value ist tensor
        value = value.numpy() # get value of tensor
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a floast_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def serialize_array(array):
  array = tf.io.serialize_tensor(array)
  return array



def parser(serialized_example):
    features_description = {'emb': tf.io.FixedLenFeature([], tf.string)}
    features = tf.io.parse_single_example(serialized_example, features_description)
    features = tf.io.decode_raw(features['emb'], tf.float32)
    return features


def parse_tfr_element(element):
    #use the same structure as above; it's kinda an outline of the structure we now want to create
    data = {
        'emb' : tf.io.FixedLenFeature([], tf.string),
        'label' : tf.io.FixedLenSequenceFeature([], tf.int64, allow_missing=True),
    }
    
    content = tf.io.parse_single_example(element, data)

    label = content['label']
    emb = content['emb']
    

    #get our 'feature'-- our image -- and reshape it appropriately
    feature = tf.io.parse_tensor(emb, out_type=tf.float32)
    return (feature, label)


def get_dataset(filename):
    #create the dataset
    dataset = tf.data.TFRecordDataset(filename)

    #pass every single feature through our mapping function
    dataset = dataset.map(
        parse_tfr_element
    )

    return dataset




In [32]:
args = pd.Series({
    "root_dir":"/mnt/disks/data/",
    "embeddings":"music_style",
    "train_id": "global_mini_test"
})

In [33]:
job_path = "/mnt/disks/data/fma/trains"

In [34]:


# In[15]:


train_path = os.path.join(job_path,args.train_id)


# In[16]:


base_path = os.path.join(args.root_dir,"fma")


# In[17]:


models_path = os.path.join(args.root_dir,"models")


# In[18]:


metadata_path = os.path.join(base_path,"fma_metadata")



In [35]:
tfrecords_path = os.path.join(train_path,'tfrecords','val')


In [36]:
os.listdir(tfrecords_path)

['0000000000.tfrecord']

In [41]:
dataset = get_dataset(os.path.join(tfrecords_path,os.listdir(tfrecords_path)[0]))


In [42]:
df = pd.DataFrame(
    dataset.as_numpy_iterator(),
    columns=['Feature', 'Label']
)

In [43]:
df

,Feature,Label
0,"[[0.010605519, -0.010418594, 0.040808935, -0.0...",[14]
1,"[[0.102527596, -0.014575779, -0.016536692, 0.0...",[57]
2,"[[-0.06529698, 0.01823314, 0.016354134, -0.014...",[16]
3,"[[-0.0683376, -0.07877615, -0.05909473, -0.109...",[1]
4,"[[-0.034052063, 0.022492925, -0.025601223, -0....",[32]
5,"[[0.2035376, 0.06256624, -0.0010401508, -0.009...",[1]
6,"[[-0.02758585, -0.030512908, -0.02157493, -0.0...",[77]
7,"[[-0.032253653, -0.00087647635, 0.03205068, 0....",[65]
8,"[[0.013740872, -0.009711902, -0.020222167, 0.0...",[43]
9,"[[0.19191581, 0.0042256415, 0.02487562, -0.019...",[1]


In [ ]:

for sample in dataset.take(1):
    print(sample[0].shape)
    print(sample[1].shape)
    print(sample)

In [29]:
filenames = os.path.join(tfrecords_path,os.listdir(tfrecords_path)[0])
raw_dataset = tf.data.TFRecordDataset(filenames)
raw_dataset

<TFRecordDatasetV2 element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [30]:
for raw_record in raw_dataset.take(3):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    print(example)

features {
  feature {
    key: "emb"
    value {
      bytes_list {
        value: "\010\001\022\t\022\002\010\035\022\003\010\200\n\"\200\210\t\305\302-<\300\262*\274E\'\'=k\272\352\273\353@k<P\346`<\3234\346\273\314\336==\363\311\035=U\331\034\274Mpl=\353\2511\273\345J\265<B\230\220=`\211\342\274\230\302\316<\353r\021\275\253\243o<|\327J\275c\356->UQ\304;\255#\330=+S3<\273\002v=U\351\361\274\003\322\002>\203@d\275\r\2574\275/\252\276=\333T\350\274_Q\343=U\020\037<\343P\231\275\237\213\202=e^ =\033\000\343<k\340\365;U$#\275\201\024\234=\325!T\274\213\021\010=\340\205]=+q\303\274\374\017\360={\335G\274\005i\026<\326Q\024=5\351\026>\245\333\246<\325\017\203\273LMi\275\025P\237\274\0315\252=\265j\205=\025_\234=\033bn\2758 \203<\325\340S;\3536\227\274I\211\270=\263\304t=\205\316\343\274\2530\250\273\240\2121<\300J\005<\217\024\215\275+\2001<\345\006\216<U\245\261<\221wX\275[\240\200>\200\377\266\274\313\240C\275\000\013\213;\360In\274\314\314l=\260\310\354;\233\233D\274U\215\204;\230\357